# 🧬 Cálculo de Fingerprints Moleculares

**Fingerprints moleculares** são representações digitais que traduzem a estrutura química de uma molécula em vetores binários ou numéricos. Eles capturam informações sobre a presença ou ausência de certos padrões estruturais, grupos funcionais ou subestruturas químicas.

---

## Para que servem os fingerprints?

- 💡 Facilitar a comparação rápida entre moléculas  
- 🔍 Realizar buscas por similaridade em grandes bancos de dados químicos  
- 🧠 Alimentar algoritmos de machine learning para tarefas como previsão de propriedades químicas, atividade biológica ou toxicidade  
- 📊 Classificação, agrupamento e análise estatística de compostos

---

## Tipos comuns de fingerprints

- **Morgan (circular)**: baseado em átomos e seus vizinhos até um raio específico  
- **Topological (path-based)**: caminhos lineares de átomos na molécula  
- **MACCS keys**: conjunto fixo de subestruturas predefinidas  
- **Avalon, Atom Pair, etc.**: outros métodos com diferentes enfoques

---

## Como calcular fingerprints com RDKit?

O RDKit oferece funções prontas para gerar fingerprints a partir de moléculas representadas em SMILES ou objetos Mol, permitindo que você obtenha vetores que podem ser usados diretamente em análises e modelos.

---


# Passo 1: Instalar a biblioteca Padelpy

A **Padelpy** é uma biblioteca Python que serve como interface para o **PaDEL-Descriptor**, um software amplamente usado para calcular descritores moleculares e fingerprints.

---

## Por que usar Padelpy?

- Permite calcular centenas de descritores moleculares e fingerprints  
- Suporta múltiplos formatos químicos (SMILES, SDF, etc.)  
- Integração simples com Python e análise de dados  
- Ferramenta poderosa para química computacional e machine learning


In [2]:
# 🛠️ Instalando a biblioteca Padelpy
! pip install padelpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 109.3 MB/s eta 0:00:00


###**Step 2: Prepare the fingerprint.xml file**

In [3]:
# Baixar o arquivo fingerprints_xml.zip do GitHub usando wget
!wget https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip

# Extrair o conteúdo do arquivo ZIP para o diretório atual
!unzip fingerprints_xml.zip


--2025-07-24 12:03:29--  https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip [following]
--2025-07-24 12:03:29--  https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10871 (11K) [application/zip]
Saving to: ‘fingerprints_xml.zip’

fingerprints_xml.zi 100%[===================>]  10.62K  --.-KB/s    in 0s      

2025-07-24 12:03:29 (150 MB/s) - ‘fingerprints_xml.zip’ saved [10871/10871]

Archive:  fingerprints_xml

# 📂 Listar os arquivos XML no diretório atual

Após extrair os arquivos do ZIP, podemos listar todos os arquivos XML presentes para verificar quais arquivos de definição de fingerprints estão disponíveis.


In [4]:
# Importar biblioteca para manipulação de arquivos
import glob

# Procurar todos os arquivos com extensão .xml no diretório atual
arquivos_xml = glob.glob("*.xml")

# Ordenar a lista para facilitar a visualização
arquivos_xml.sort()

# Exibir a lista de arquivos XML encontrados
arquivos_xml


['AtomPairs2DFingerprintCount.xml',
 'AtomPairs2DFingerprinter.xml',
 'EStateFingerprinter.xml',
 'ExtendedFingerprinter.xml',
 'Fingerprinter.xml',
 'GraphOnlyFingerprinter.xml',
 'KlekotaRothFingerprintCount.xml',
 'KlekotaRothFingerprinter.xml',
 'MACCSFingerprinter.xml',
 'PubchemFingerprinter.xml',
 'SubstructureFingerprintCount.xml',
 'SubstructureFingerprinter.xml']

# 📋 Lista dos Tipos de Fingerprints a serem calculados

Aqui definimos uma lista com os nomes dos fingerprints que desejamos calcular utilizando o PaDEL-Descriptor via Padelpy. Cada tipo representa um método diferente de codificar informações estruturais das moléculas.

---

## Tipos de Fingerprints incluídos na lista

- **AtomPairs2DCount**: Contagem de pares de átomos em 2D  
- **AtomPairs2D**: Fingerprints baseados em pares de átomos em 2D  
- **EState**: Índices eletrônicos de estado dos átomos  
- **CDKextended**: Fingerprints estendidos do Chemistry Development Kit (CDK)  
- **CDK**: Fingerprints padrão do CDK  
- **CDKgraphonly**: Fingerprints baseados apenas no grafo molecular do CDK  
- **KlekotaRothCount**: Contagem dos fragmentos definidos por Klekota e Roth  
- **KlekotaRoth**: Fingerprints dos fragmentos Klekota e Roth  
- **MACCS**: Chaves MACCS clássicas (166 bits)  
- **PubChem**: Fingerprints usados pelo banco PubChem  
- **SubstructureCount**: Contagem de subestruturas químicas específicas  
- **Substructure**: Presença/ausência de subestruturas químicas específicas  

---


In [5]:
lista_FP = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

# 🔗 Associação dos nomes dos fingerprints aos arquivos XML

Neste passo, vamos criar um dicionário Python que mapeia cada nome de fingerprint da lista para seu arquivo XML correspondente extraído anteriormente. Isso facilita a escolha do arquivo correto para cada tipo de fingerprint na hora do cálculo.


In [6]:
# Criar um dicionário que associa os nomes dos fingerprints aos arquivos XML
fp = dict(zip(lista_FP, arquivos_xml))

# Exibir o dicionário resultante
fp


{'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'MACCS': 'MACCSFingerprinter.xml',
 'PubChem': 'PubchemFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml',
 'Substructure': 'SubstructureFingerprinter.xml'}

# 🔍 Acessando o arquivo XML do fingerprint "PubChem"

Usamos o dicionário `fp` para obter o nome do arquivo XML correspondente ao fingerprint “PubChem”. Isso facilita o uso desse arquivo para calcular os fingerprints específicos desse tipo.


In [7]:
# Mostrar o nome do arquivo XML que corresponde ao fingerprint "PubChem"
fp['PubChem']


'PubchemFingerprinter.xml'

# 📥 Importando o Dataset Tratado

Nesta etapa, vamos carregar um arquivo de dados previamente tratado e salvo (por exemplo, um CSV com SMILES validados e padronizados) para continuar com a análise ou cálculo de descritores moleculares.


In [37]:
# Importar a função para upload de arquivos no Google Colab
from google.colab import files

# Abrir uma janela para selecionar e enviar arquivos do seu computador
uploaded = files.upload()


In [38]:
# Importar a biblioteca pandas para manipulação de dados
import pandas as pd

# Carregar o arquivo CSV no DataFrame 'df'
df = pd.read_csv("/content/validacao_smiles_sem_duplicatas_AFRICA.csv")

# Exibir o conteúdo do DataFrame para visualização inicial
df


,SMILES,canonical_smiles,SMILES_Canonico
0,COC1=CC=C2CN(C)CC[C@@]34C=C[C@H](O)C[C@@H]3OC1...,COc1ccc2c3c1O[C@H]1C[C@@H](O)C=C[C@@]31CCN(C)C2,COc1ccc2c3c1O[C@H]1C[C@@H](O)C=C[C@@]31CCN(C)C2
1,CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C...,CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C...,CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C...
2,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...
3,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](OC3=C(C4=CC=C(...,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(-c4ccc(O)...,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(-c4ccc(O)...
4,C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CCC4=C3C=C[...,C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CCC4=C3C=C[...,C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CCC4=C3C=C[...
...,...,...,...
5779,C=C1C(=O)O[C@@H]2C1CC[C@H](C)[C@@H]1CC(=O)C(=O...,C=C1C(=O)O[C@@H]2C1CC[C@H](C)[C@@H]1CC(=O)C(=O...,C=C1C(=O)O[C@@H]2C1CC[C@H](C)[C@@H]1CC(=O)C(=O...
5780,COC1=CC(=O)[C@@H](Cl)C(O)[C@@H]1O,COC1=CC(=O)[C@@H](Cl)C(O)[C@@H]1O,COC1=CC(=O)[C@@H](Cl)C(O)[C@@H]1O
5781,C=C(C)[C@@H]1CC[C@]2(C)[C@@H](CC[C@H]3C=CC[C@]...,C=C(C)[C@@H]1CC[C@]2(C)[C@@H](CC[C@H]3C=CC[C@]...,C=C(C)[C@@H]1CC[C@]2(C)[C@@H](CC[C@H]3C=CC[C@]...
5782,O=C1C(OC2OC[C@@H](O)[C@@H](O)[C@@H]2O)=C(C2=CC...,O=c1c(OC2OC[C@@H](O)[C@@H](O)[C@@H]2O)c(-c2ccc...,O=c1c(OC2OC[C@@H](O)[C@@H](O)[C@@H]2O)c(-c2ccc...


# 📝 Criar arquivo `.smi` para uso com PaDEL-Descriptor

O arquivo `.smi` contém moléculas em formato SMILES e seus identificadores. Neste caso, usamos o próprio SMILES como identificador, já que é único após padronização.


In [39]:
# Importar a função para calcular descritores moleculares com PaDEL
from padelpy import padeldescriptor

# Definir o fingerprint desejado (exemplo: 'PubChem')
fingerprint = 'PubChem'

# Definir o caminho do arquivo CSV que armazenará os resultados
fingerprint_output_file = f"{fingerprint}.csv"

# Obter o arquivo XML que define o fingerprint a partir do dicionário 'fp'
fingerprint_descriptortypes = fp[fingerprint]


# ⚙️ Gerar descritores moleculares usando PaDEL-Descriptor

Usamos a função `padeldescriptor` da biblioteca `padelpy` para calcular fingerprints a partir do arquivo `.smi` com SMILES e IDs, especificando parâmetros para padronização e desempenho.


In [40]:
# Create the molecule.smi file using 'smille' as both SMILES and identifier
df[['SMILES_Canonico']].to_csv('molecule.smi', sep='\t', index=False, header=False)

# Re-read and duplicate the column so SMILES is used as its own ID
with open('molecule.smi', 'r') as infile, open('molecule_smi_with_ids.smi', 'w') as outfile:
    for line in infile:
        smi = line.strip()
        outfile.write(f"{smi}\t{smi}\n")  # SMILES \t SMILES as ID

In [41]:
from padelpy import padeldescriptor

# Executar o cálculo dos descritores/fingerprints usando o arquivo com SMILES e IDs
padeldescriptor(
    mol_dir='molecule_smi_with_ids.smi',        # Arquivo de entrada com SMILES e IDs
    d_file=fingerprint_output_file,             # Arquivo CSV de saída para os resultados
    descriptortypes=fingerprint_descriptortypes,  # Arquivo XML com o tipo de fingerprint a calcular
    detectaromaticity=True,                      # Detectar aromaticidade durante o cálculo
    standardizenitro=True,                       # Padronizar grupos nitro
    standardizetautomers=True,                   # Padronizar formas tautoméricas
    threads=2,                                   # Número de threads para acelerar o processamento
    removesalt=True,                             # Remover sais antes do cálculo
    log=True,                                    # Ativar logs detalhados da execução
    fingerprints=True                            # Indica que queremos gerar fingerprints
)


# 📂 Carregar e visualizar os fingerprints gerados pelo PaDEL

Após o cálculo dos fingerprints, os resultados são salvos em um arquivo CSV. Nesta etapa, vamos carregar esse arquivo para análise e manipulação usando o pandas.


In [65]:
# Importar a biblioteca pandas para manipulação de dados
import pandas as pd

# Carregar o arquivo CSV com os fingerprints gerados (exemplo: 'PubChem.csv')
df_fp = pd.read_csv('PubChem.csv')

# Exibir as primeiras linhas do DataFrame para inspeção
df_fp.head()


,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,COc1ccc2c3c1O[C@H]1C[C@@H](O)C=C[C@@]31CCN(C)C2,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(-c4ccc(O)...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CCC4=C3C=C[...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 🔍 Comparação do número de moléculas antes e depois do cálculo dos fingerprints

Vamos verificar quantas moléculas estavam no arquivo SMILES original e quantas resultaram no arquivo CSV gerado pelo PaDEL para garantir que não houve perda de dados.


In [66]:
# Garantir que índices estão alinhados
df_fp = df_fp.reset_index(drop=True)
df = df.reset_index(drop=True)

# Remove the existing 'Name' column from df_fp
if 'Name' in df_fp.columns:
    df_fp = df_fp.drop('Name', axis=1)

# Fill any potential NaN values with 0 before converting to integer
# Fingerprint data is typically binary (0 or 1), so 0 is a reasonable placeholder for missing values.
df_fp = df_fp.fillna(0)

# Convert remaining columns to integers
df_fp = df_fp.astype(int)

# Inserir a coluna 'Name' com os SMILES canônicos
df_fp.insert(0, 'Name', df['SMILES_Canonico'])

# Visualizar o DataFrame final
display(df_fp)

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,COc1ccc2c3c1O[C@H]1C[C@@H](O)C=C[C@@]31CCN(C)C2,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C...,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(-c4ccc(O)...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CCC4=C3C=C[...,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779,C=C1C(=O)O[C@@H]2C1CC[C@H](C)[C@@H]1CC(=O)C(=O...,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5780,COC1=CC(=O)[C@@H](Cl)C(O)[C@@H]1O,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5781,C=C(C)[C@@H]1CC[C@]2(C)[C@@H](CC[C@H]3C=CC[C@]...,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5782,O=c1c(OC2OC[C@@H](O)[C@@H](O)[C@@H]2O)c(-c2ccc...,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
df_fp

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,COc1ccc2c3c1O[C@H]1C[C@@H](O)C=C[C@@]31CCN(C)C2,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C...,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(-c4ccc(O)...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CCC4=C3C=C[...,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779,C=C1C(=O)O[C@@H]2C1CC[C@H](C)[C@@H]1CC(=O)C(=O...,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5780,COC1=CC(=O)[C@@H](Cl)C(O)[C@@H]1O,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5781,C=C(C)[C@@H]1CC[C@]2(C)[C@@H](CC[C@H]3C=CC[C@]...,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5782,O=c1c(OC2OC[C@@H](O)[C@@H](O)[C@@H]2O)c(-c2ccc...,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
# Contar o número de linhas no arquivo molecule.smi (número de moléculas originais)
with open("molecule.smi") as f:
    original_count = sum(1 for line in f)
print("Contagem original de SMILES:", original_count)

# Importar pandas para ler o arquivo CSV gerado pelo PaDEL
import pandas as pd
df_fp = pd.read_csv("PubChem.csv")

# Mostrar o número de linhas no arquivo de saída do PaDEL
print("Contagem de moléculas no arquivo PaDEL:", df_fp.shape[0])



Contagem original de SMILES: 5784
Contagem de moléculas no arquivo PaDEL: 5784


# 💾 Salvar o DataFrame de fingerprints em arquivo CSV

Após a geração e análise dos fingerprints, podemos salvar o DataFrame resultante em um arquivo CSV para uso posterior ou compartilhamento.


In [68]:
# Salvar o DataFrame 'df_fp' em um arquivo CSV chamado "Base de dados moleculas aprovadas.csv"
df_fp.to_csv("Base_Pubchem_AFRICA.csv", index=False)


# Referência

A base para os arquivos de definição de fingerprints moleculares utilizados no cálculo com o software **PaDEL-Descriptor** foi obtida do repositório do Data Professor no GitHub:

- **Arquivo:** [fingerprints_xml.zip](https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip)  
- **Descrição:** Pacote contendo arquivos XML que configuram os tipos de fingerprints e descritores moleculares que podem ser calculados pelo PaDEL-Descriptor  
- **Importância:** Essencial para customizar os cálculos de descritores/fingerprints via a biblioteca Python **Padelpy**, garantindo flexibilidade e precisão nas análises químicas computacionais  

---

Este recurso é fundamental para quem deseja utilizar o PaDEL-Descriptor em pipelines de química computacional e machine learning, permitindo o controle detalhado sobre os parâmetros e tipos de fingerprints gerados.
